# Homework2
Author: Yichen Xie (N13905325)

Date: 2/10/2022

### Fetch Data from sklearn

In [188]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups = fetch_20newsgroups(subset='test')

# getting the topic ids
topic_idx = np.array(newsgroups.target, dtype=int)

# getting the unique topic names
topic_names = np.array(newsgroups.target_names)

# getting the list of documents
documents = list(newsgroups.data)

# getting the list of topics (in the same order as documents)
topics = list(topic_names[topic_idx])

# total entries
N = len(documents)

print("Total documents:",N)
print("Topic names:",topic_names)

Total documents: 7532
Topic names: ['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


### Functions for processing data

In [189]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# filter out the unwanted characters
def getWords(string):
    string = string.lower()
    for char in ':!"$&()*+,-./<=>?@[\\]^_{|}~`1234567890%#\';':
        string = string.replace(char," ")
    return string

# find the topk words in given topic list
def getData(top_k):
    data = []
    word_count = {}
    stop_words = set(stopwords.words('english'))

    word_count['All'] = {}
    for t in topic_names:
        word_count[t] = {}
    for i in range(N):
        cur_topic = topics[i]
        word_array = getWords(documents[i]).split()
        for w in word_array:
            if not w in stop_words:
                word_count[cur_topic][w] = word_count[cur_topic].get(w,0) + 1
                word_count['All'][w] = word_count['All'].get(w,0) + 1


    allitems = list(word_count['All'].items())
    allitems.sort(key = lambda x:x[1], reverse=True)
    for i in range(top_k):
        data.append({'word':allitems[i][0],'count':allitems[i][1],'topic':'All'})

    for t in topic_names:
        items = list(word_count[t].items())
        items.sort(key = lambda x:x[1], reverse=True)

        for i in range(top_k):
            data.append({'word':items[i][0],'count':items[i][1],'topic':t})

    return data


# function used for callback, get documents from selected topic
result = []

def getDocuments(data):
    selectedtopic = data['t']
    retdocuments = []
    global result
    for i in range(N):
        if topics[i] == selectedtopic or selectedtopic == 'All':
            retdocuments.append(documents[i])
    result = retdocuments

# Sample   
len(getData(10))


210

### Import Javascript Libs

In [190]:
from notebookjs import execute_js
import glob
import pandas as pd
import random
import json

### Draw Bar Chart by D3

In [191]:
d3_lib_url = "https://d3js.org/d3.v3.min.js"
    
with open ("./bar_chart.js", "r") as f:
    bar_lib = f.read()

topk = 10

selected_data = getData(topk)

execute_js(library_list=[d3_lib_url, bar_lib], 
            main_function="bar_chart", 
            data_dict=selected_data, 
            callbacks={"getDocuments":getDocuments})

### Get seleted topic doucments
The variable **result** contains documents strings from selected topic. You can see the changes by running this part again after click the "Export Document" button above. 

In [192]:
len(result)

0

### Final APIs
Just a wrapper XD

In [193]:
def plot_top_words(documents, K):
    selected_data = getData(K)
    execute_js(library_list=[d3_lib_url, bar_lib], 
                main_function="bar_chart", 
                data_dict=selected_data, 
                callbacks={"getDocuments":getDocuments})

In [194]:
def get_exported_documents():
    return result